# <center> Python Client Example
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 

### Prequisite
Make sure [Tornado Server Example.ipynb](Tornado%20Server%20Example.ipynb) is running. 

Suggestion:  Split view the windows so you can see both this notebook and [Tornado Server Example.ipynb](Tornado%20Server%20Example.ipynb)

# Web Clients
While often we think of a web client to be a browser, we can also use programs or libraries to access web based resources. 

Python has the requests library for these transactions.

https://requests.readthedocs.io/en/master/

Run this notebook after the Tornado server notebook is running.

In [1]:
import requests
import base64
import json

In [2]:
# Once the Tornado Server Example is running, execute this code.
url = "http://localhost:8080"
r = requests.get(url)
print(r)

<Response [200]>


What are the status codes?

https://developer.mozilla.org/en-US/docs/Web/HTTP/Status

200 is good.

In [3]:
#Responses have status codes
r.status_code

200

In [4]:
# Text base response
r.text

'{"message": "The current time is Mon Apr 21 20:50:37 2025"}'

In [5]:
# Bytes as the response. Use this to read binary (like photos)
r.content 

b'{"message": "The current time is Mon Apr 21 20:50:37 2025"}'

In [6]:
# Access lower level bytes from the socket
r = requests.get(url, stream=True)
r.raw

In [7]:
r.raw.read()

b'{"message": "The current time is Mon Apr 21 20:50:37 2025"}'

In [8]:
# Headers are often not displayed to the client, but are available
# requests makes it a dictionary. 
# This is the header response from the server
r.headers

{'Server': 'TornadoServer/6.4.1', 'Content-Type': 'text/html; charset=UTF-8', 'Date': 'Tue, 22 Apr 2025 02:50:37 GMT', 'Etag': '"6c2f4c2b5d50c24779bf363dc97fbed43879dd5d"', 'Content-Length': '59'}

In [9]:
r.encoding

'UTF-8'

In [10]:
r.url

'http://localhost:8080/'

In [11]:
#This is actualy json, so we can load it directly into a dictionary
r = requests.get(url)
print(r.status_code)
r.json()

200


{'message': 'The current time is Mon Apr 21 20:50:37 2025'}

In [12]:
#This is actualy json, so we can load it directly into a dictionary
r = requests.get(url+"/encrypted/")
print(r.status_code)
r.json()

200


{'client_id': '9b6a676f-48b4-4180-8511-5cb89483d1fc',
 'pub': '-----BEGIN PUBLIC KEY-----\nMCowBQYDK2VuAyEAVO71C8WDD7TWhd1gO4Lc0P638ktxMTKg6/rcpeWCxDw=\n-----END PUBLIC KEY-----\n'}

In [13]:
#Remember the client_id from the server so we can get the right keys
client_id = r.json()['client_id']
client_id

'9b6a676f-48b4-4180-8511-5cb89483d1fc'

In [14]:
plain_text = "I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character."
plain_text

'I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character.'

In [15]:
# This is a bad request
r = requests.post(url+"/encrypted/", 
                  json={'plain_text':plain_text })
print(r)
print(r.text)

<Response [400]>
{"error": "Missing or unknown client_id"}


In [16]:
# This is another bad request
r = requests.post(url+"/encrypted/", 
                  json={'plain_text':plain_text, 'client_id':client_id })
print(r)
print(r.text)

<Response [400]>
{"error": "Bad request. Expected cipher_text or pub_key"}


In [17]:
# This is caused the server to crash
r = requests.post(url+"/encrypted/", 
                  json={'cipher_text':plain_text, 'client_id':client_id })
print(r)
print(r.text)

<Response [500]>
Traceback (most recent call last):
  File "C:\Users\jdaily\anaconda3\Lib\site-packages\tornado\web.py", line 1788, in _execute
    result = method(*self.path_args, **self.path_kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jdaily\AppData\Local\Temp\ipykernel_20416\2090479051.py", line 52, in post
    print(f"Using session key: {shared_secrets[client_id]}")
                                ~~~~~~~~~~~~~~^^^^^^^^^^^
KeyError: '9b6a676f-48b4-4180-8511-5cb89483d1fc'



## Why You Should Never Return Tracebacks to the Client

Returning full Python error tracebacks in an HTTP response can expose sensitive internal details to attackers. Here's why it's dangerous:

### 1. Reveals Internal Code Structure
Tracebacks show:
- File paths
- Function names
- Variable names
- Line numbers

This gives attackers a **blueprint of your application**, making it easier to exploit.

### 2. Enables Targeted Attacks
By seeing exactly where and how your server failed, attackers can:
- Craft smarter input payloads
- Bypass validation
- Exploit known vulnerabilities

### 3. May Leak Sensitive Data
Some exceptions can expose:
- API keys
- Tokens
- SQL queries
- Internal system details

Never return this kind of data to the client.

### 4. Violates the Principle of Least Information
Clients should only get minimal, non-sensitive feedback:
```json
{ "error": "Invalid input" }


### Exercise
Please find the endpoint logic in [Tornado Server Example.ipynb](Tornado%20Server%20Example.ipynb) and wrap the handler for `cipher_text` in a try, except block and return a Status Code 400 with minimal information. 

Bonus: add logging to the server so the engineering team can see what's going on. 
```python
import logging

try:
    # risky code
except Exception as e:
    logging.exception("Internal server error")  # logs traceback safely
    self.set_status(500)
    self.write({ "error": "Internal server error" })  # safe, generic message
```

In [18]:
#Get the key (Don't actually do this without verification.)
r = requests.get(url+"/encrypted/")
print(r.status_code)
print(r.text)

200
{"client_id": "7a7f5b19-b77b-4b65-afd9-c1054f5ee849", "pub": "-----BEGIN PUBLIC KEY-----\nMCowBQYDK2VuAyEAkVMnhCHPriKTN4y4bGJ51dDdM22e9EQvVAnqsxsnJBs=\n-----END PUBLIC KEY-----\n"}


In [19]:
#Remember the client_id from the server so we can get the right keys
client_id = r.json()['client_id']
client_id

'7a7f5b19-b77b-4b65-afd9-c1054f5ee849'

In [20]:
#Extract the public PEM key
pub_key_text = r.json()['pub']
print(pub_key_text)

-----BEGIN PUBLIC KEY-----
MCowBQYDK2VuAyEAkVMnhCHPriKTN4y4bGJ51dDdM22e9EQvVAnqsxsnJBs=
-----END PUBLIC KEY-----



## Is It Safe to Get a Public Key Over the Internet?

Yes — retrieving a public key over the open internet is generally acceptable, since public keys are meant to be shared. However, there are important security considerations to keep in mind.

### Why It's Okay
Public keys are used to:
- Encrypt data that only the private key holder can decrypt
- Verify digital signatures made with the corresponding private key

So exposing a public key via an endpoint like:
```python
r = requests.get("http://example.com/public-key")
```
is acceptable as long as the **key's authenticity** can be verified.

### Risks Without Verification

Even though public keys aren’t secret, an attacker could:

- **Intercept the connection** (if not encrypted)  
- **Replace the key with a forged one**  

This enables:

- Man-in-the-middle (MITM) attacks  
- Spoofed signatures or encrypted messages sent to the attacker  

### Best Practices for Public Key Distribution

| Practice                     | Why It Matters                                             |
|-----------------------------|------------------------------------------------------------|
| **Use HTTPS**               | Prevents tampering during transmission                     |
| **Pin Key Fingerprints**    | Ensure the received key matches a known trusted hash       |
| **Sign the Key**            | Include a digital signature from a trusted authority (i.e. certificates)       |
| **Distribute via Trusted Channels** | Use official websites, DNSSEC, package managers, etc.  |


In [21]:
#Bring in the X25519 elliptic curve tools used for key exchange.
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
#serialization is used to convert keys to and from PEM (text) format.
from cryptography.hazmat.primitives import serialization
#Client needs to generate a new random key pair to be used in the key exchange.
private_key_for_client= X25519PrivateKey.generate()
private_key_for_client

In [22]:
#Extract the public key from the private key and
# serialize it to PEM format so it can be safely transmitted (e.g., over HTTP).
client_public_pem = private_key_for_client.public_key().public_bytes(
       encoding=serialization.Encoding.PEM,
       format=serialization.PublicFormat.SubjectPublicKeyInfo
)
print(client_public_pem)

b'-----BEGIN PUBLIC KEY-----\nMCowBQYDK2VuAyEA162x19mIBjJygisQJt8U2VfXWbzkVzJaUhiQklgf8Go=\n-----END PUBLIC KEY-----\n'


In [23]:
# Convert the server’s PEM-encoded public key string into a usable object.
# Assume pub_key_text came from an HTTP response or local file.
server_pub_key = serialization.load_pem_public_key(pub_key_text.encode('ascii'))
server_pub_key

In [24]:
# Use the client’s private key and the server’s public key to compute a shared key.
shared_key = private_key_for_client.exchange(server_pub_key)
shared_key

b'\xe1\tDn\xd7,\x05\xcc\xb2\x0c\xbb\x14:x\x08\nF\x13\x88m\xd58\xa4\x15\xd38\xa2\xa0\xe0\xff\x9a\x1a'

This client has a shared secret, but the server does not. We need to send the server our public key so it can derive the same shared secret. Then we can use the shared secret to exchange information or setup session keys.

In [25]:
# Send the server our public key so it can calulate a secret key
r = requests.post(url+"/encrypted/", json={
        'pub_key':client_public_pem.decode('utf-8'),
        'uid':1, 'client_id':client_id 
    })
print(r.status_code)
r.json()

200


{'message': 'Shared secret successfully calculated'}

Now both the client and server application have an ephemeral session key that is secret by only trading public keys.

In [26]:
# Let's setup a symmetric cipher to share information at the application level.
from cryptography.fernet import Fernet
session_cipher = Fernet(base64.urlsafe_b64encode(shared_key))
session_cipher

In [27]:
cipher_text = session_cipher.encrypt(plain_text.encode('utf-8'))
cipher_text

b'gAAAAABoBwP-K541aRKBk-O6T_gp5QUoe7RsoQEMLXciKc1ueJEUYW9zlhVdC2IvFSVWWA-hYnF9fJ0ow-L7lY81zqnSvG5T7bRo6fSyjS1KnCV7VQqYlr4JlSOWs-TVICP0LHZJLSA3IGqiBYV-1Ad8TLEX5tjuTkzZrH_MzED38PawNEvakJtQpEb1o_2EL7eOsAImf7VsEf_Rg7g0V4tJld_YA-jSGMiAt14aCO3C9cr8OUSBC5piZ5eJ0n7RDH9X2HWHukpUGjt3wrpXlbJCioMTB_xLYm9twxz0GcgvFgWvwrreNjM='

In [28]:
r = requests.post(url+"/encrypted/", 
                  json={'cipher_text':cipher_text.decode('utf-8'), 'client_id':client_id  })
print(r)
print(r.text)

<Response [200]>
{"message": "Successfully Decrypted Data"}


## Summary
* We can see how data can be encrypted and sent across the Internet. When using symmetric encryption, there needs to be a key exchange. This should never be done in the open, like we did in this example. 

* We introduced GET and POST

* We introduced JSON

* We talked about HTTP status codes

In [29]:
#Unicode Character fun
u"\U0001F44F"

'👏'